In [1]:
import numpy as np

### Q0)

In [10]:
# Load data
data = np.genfromtxt('meteo0.csv', dtype='int')
N, T = data.shape

In [11]:
# Compute initial probability
_,pi = np.unique(data[:, 0], return_counts = True)
pi = pi / np.sum(pi)

# Compute transition probability
A = np.zeros((3,3))
for n in range(N):
    seq = data[n, :]
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            A[i,j] += np.sum((seq[1:] == i) * (seq[:-1] == j))
            
A = A / np.sum(A, axis = 0).reshape(1, -1)
print("pi")
print(pi)
print("A")
print(A)

pi
[0.308 0.512 0.18 ]
A
[[0.69876424 0.10079219 0.10071453]
 [0.30123576 0.6995032  0.19870704]
 [0.         0.19970461 0.70057843]]


### Q1)

In [15]:
data = np.genfromtxt('meteo1.csv', dtype='int')
N,T = data.shape

In [58]:
# EM Algo
# 1. Initialise the parameters
sigma = np.random.uniform(0, 1, size = (3))
sigma = sigma / np.sum(sigma)

pi = np.random.uniform(0, 1, size = (3,3))
pi = pi / np.sum(pi, axis = 0).reshape(1, -1)

A = np.random.uniform(0, 1, size = (3,3,3))
A = A / np.sum(A, axis = 0).reshape(1, 3, 3)

In [59]:
# EM Algo
import copy

for it in range(30):
    # Iterate until limit reached or no change
    old_A = copy.deepcopy(A); old_pi = copy.deepcopy(pi); old_sigma = copy.deepcopy(sigma)

    # 2. E -step. Computed in terms of log first to prevent overflow
    q = np.zeros((N, 3))

    for n in range(N):
        seq = data[n, :]; v1 = seq[0]
        vt = seq[1:]; vt_m1 = seq[:-1]
        q[n, :] = np.log(sigma) + np.log(pi[v1, :]) + np.sum(np.log(A[vt, vt_m1, :]), axis = 0)

    # Scale before normalize over h
    q -= np.max(q, axis = 1).reshape(-1, 1)

    # Normalize
    q_all = np.exp(q)
    q_all = q_all / np.sum(q_all, axis = 1).reshape(-1, 1)

    #3. M-step. Compute new parameters

    # sigma
    sigma = np.sum(q_all, axis = 0) / np.sum(q_all)

    # pi
    for i in range(pi.shape[0]):
        pi[i, :] = np.sum((data[:, 0] == i).reshape(-1, 1) * q_all, axis = 0)

    pi = pi / np.sum(pi, axis = 0).reshape(1, -1)

    # A
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            transitions = (data[:, 1:] == i) * (data[:, :-1] == j)
            counts = np.sum(transitions, axis = 1)
            A[i,j, :] = np.sum(counts.reshape(-1, 1) * q_all, axis = 0)

    A = A / np.sum(A, axis = 0)
    
    # Total error
    change = np.sum(np.abs(A - old_A)) + np.sum(np.abs(pi - old_pi)) + np.sum(np.abs(sigma - old_sigma))
    print("Total change ", change)
    if change < 1e-10:
        print(f"Finished at iteration {it+1}")
        break

Total change  6.076295493077287
Total change  1.0874862676264305
Total change  0.8192066043340482
Total change  0.21320008541369878
Total change  0.019746787917213324
Total change  0.0019088945272602437
Total change  0.00018026749421437907
Total change  1.6790627717557655e-05
Total change  1.557230424758388e-06
Total change  1.4420148928412369e-07
Total change  1.3345798001440645e-08
Total change  1.2349076483619825e-09
Total change  1.1425858176261627e-10
Total change  1.05730701971396e-11
Finished at iteration 14


In [60]:
# Print per-station
for i in range(3):
    print(f"For station {i+1} \n")
    print("sigma = ", sigma[i])
    print("Start prob")
    print(pi[:, i], "\n")
    print("Transition Prob")
    print(A[:, :, i], '\n')
    print("\n")

For station 1 

sigma =  0.5083010606295831
Start prob
[0.46117125 0.33120302 0.20762573] 

Transition Prob
[[0.07051106 0.13676591 0.5107758 ]
 [0.49465443 0.22585642 0.43723563]
 [0.43483451 0.63737768 0.05198857]] 



For station 2 

sigma =  0.2996989393812316
Start prob
[0.19214671 0.41925124 0.38860204] 

Transition Prob
[[0.05971003 0.13007012 0.41759899]
 [0.13950709 0.32378127 0.42784294]
 [0.80078288 0.54614861 0.15455807]] 



For station 3 

sigma =  0.1919999999891857
Start prob
[4.27083333e-001 5.72916667e-001 7.60475509e-210] 

Transition Prob
[[0.38869863 0.2405914  0.54531568]
 [0.14848337 0.51545699 0.01756619]
 [0.462818   0.24395161 0.43711813]] 





In [61]:
# If uniform init
sigma = np.ones((3))
sigma = sigma / np.sum(sigma)

pi = np.ones((3,3))
pi = pi / np.sum(pi, axis = 0).reshape(1, -1)

A = np.ones((3,3,3))
A = A / np.sum(A, axis = 0).reshape(1, 3, 3)

In [62]:
# EM Algo
import copy

for it in range(30):
    # Iterate until limit reached or no change
    old_A = copy.deepcopy(A); old_pi = copy.deepcopy(pi); old_sigma = copy.deepcopy(sigma)

    # 2. E -step. Computed in terms of log first to prevent overflow
    q = np.zeros((N, 3))

    for n in range(N):
        seq = data[n, :]; v1 = seq[0]
        vt = seq[1:]; vt_m1 = seq[:-1]
        q[n, :] = np.log(sigma) + np.log(pi[v1, :]) + np.sum(np.log(A[vt, vt_m1, :]), axis = 0)

    # Scale before normalize over h
    q -= np.max(q, axis = 1).reshape(-1, 1)

    # Normalize
    q_all = np.exp(q)
    q_all = q_all / np.sum(q_all, axis = 1).reshape(-1, 1)

    #3. M-step. Compute new parameters

    # sigma
    sigma = np.sum(q_all, axis = 0) / np.sum(q_all)

    # pi
    for i in range(pi.shape[0]):
        pi[i, :] = np.sum((data[:, 0] == i).reshape(-1, 1) * q_all, axis = 0)

    pi = pi / np.sum(pi, axis = 0).reshape(1, -1)

    # A
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            transitions = (data[:, 1:] == i) * (data[:, :-1] == j)
            counts = np.sum(transitions, axis = 1)
            A[i,j, :] = np.sum(counts.reshape(-1, 1) * q_all, axis = 0)

    A = A / np.sum(A, axis = 0)
    
    # Total error
    change = np.sum(np.abs(A - old_A)) + np.sum(np.abs(pi - old_pi)) + np.sum(np.abs(sigma - old_sigma))
    print("Total change ", change)
    if change < 1e-10:
        print(f"Finished at iteration {it+1}")
        break

Total change  4.328952008091643
Total change  0.0
Finished at iteration 2


In [63]:
# Print per-station
for i in range(3):
    print(f"For station {i+1} \n")
    print("sigma = ", sigma[i])
    print("Start prob")
    print(pi[:, i], "\n")
    print("Transition Prob")
    print(A[:, :, i], '\n')
    print("\n")

For station 1 

sigma =  0.33333333333333404
Start prob
[0.374 0.404 0.222] 

Transition Prob
[[0.15977377 0.14452926 0.48703845]
 [0.30597384 0.28346056 0.35019099]
 [0.53425239 0.57201018 0.16277056]] 



For station 2 

sigma =  0.33333333333333404
Start prob
[0.374 0.404 0.222] 

Transition Prob
[[0.15977377 0.14452926 0.48703845]
 [0.30597384 0.28346056 0.35019099]
 [0.53425239 0.57201018 0.16277056]] 



For station 3 

sigma =  0.33333333333333404
Start prob
[0.374 0.404 0.222] 

Transition Prob
[[0.15977377 0.14452926 0.48703845]
 [0.30597384 0.28346056 0.35019099]
 [0.53425239 0.57201018 0.16277056]] 





### Q2)

In [75]:
def compute_alpha_t(trans, em, init_p, data):
    N_state = trans.shape[0]; N = data.shape[0]
    # computes until data[-1]
    alphas = np.zeros((N, N_state))
    alphas[0, :] = em[data[0], :] * init_p
    alphas[0, :] = alphas[0, :] / np.sum(alphas[0, :])
    
    for t in range(1, N):
        prev_alpha = alphas[t-1, :].reshape(-1, 1)
        alphas[t, :] = ((trans @ prev_alpha) * em[data[t], :].reshape(-1, 1)).reshape(-1)
        alphas[t, :] /= np.sum(alphas[t, :])
        
    return alphas

def compute_beta_t(trans, em, init_p, data):
    N_state = trans.shape[0]; N = data.shape[0]
    # computes until data[-1]
    betas = np.ones((N, N_state))
    
    for t in range(N-2, -1, -1):
        betas[t, :] = (em[data[t+1], :].reshape(1, -1) @ (betas[t+1, :].reshape(-1, 1) * trans)).reshape(-1)
        
    return betas

def compute_pht_given_v(alphas, betas):
    numerator = alphas * betas
    result = numerator / np.sum(numerator, axis = 1).reshape(-1, 1)
    
    return result

In [90]:
# Parameters and Data
chess_sequence = np.genfromtxt('chess.txt', dtype='int')

# 2c)
sequence =  np.array([0, 1, 0, 2, 0, 2, 1, 0, 2, 0])
initProb = np.array([1, 0])
transMat = np.array([[0.5, 0.8], [0.5, 0.2]])
emiProb = np.array([[0.3, 0.5], [0.6, 0], [0.1, 0.5]])


In [76]:
alphas = compute_alpha_t(transMat, emiProb, initProb, sequence)
betas = compute_beta_t(transMat, emiProb, initProb, sequence)
result = compute_pht_given_v(alphas, betas)
print(result)

[[1.         0.        ]
 [1.         0.        ]
 [0.50239234 0.49760766]
 [0.29824561 0.70175439]
 [0.73205742 0.26794258]
 [0.17065391 0.82934609]
 [1.         0.        ]
 [0.48837209 0.51162791]
 [0.34108527 0.65891473]
 [0.59302326 0.40697674]]


In [84]:
# 2d)
def compute_phtt1_given_v(alphas, betas, trans, em, data):
    # Will only have T-1 rows, instead of T, because its t and t+1
    numerator = []
    for t in range(alphas.shape[0] - 1):
        alpha = alphas[t, :]; beta = betas[t+1, :]
        num_term = (beta.reshape(-1, 1) @ alpha.reshape(1, -1) ) * em[data[t+1], :].reshape(-1, 1) * trans
        num_term /= np.sum(num_term)
        numerator.append(num_term)
        
    return np.array(numerator)

In [87]:
result = compute_phtt1_given_v(alphas, betas, transMat, emiProb, sequence)
print(result)

[[[1.         0.        ]
  [0.         0.        ]]

 [[0.50239234 0.        ]
  [0.49760766 0.        ]]

 [[0.08133971 0.2169059 ]
  [0.42105263 0.28070175]]

 [[0.15789474 0.57416268]
  [0.14035088 0.12759171]]

 [[0.08133971 0.08931419]
  [0.6507177  0.17862839]]

 [[0.17065391 0.82934609]
  [0.         0.        ]]

 [[0.48837209 0.        ]
  [0.51162791 0.        ]]

 [[0.09302326 0.24806202]
  [0.39534884 0.26356589]]

 [[0.12790698 0.46511628]
  [0.21317829 0.19379845]]]


In [88]:
def compute_sc_beta_t(trans, em, init_p, data):
    N_state = trans.shape[0]; N = data.shape[0]
    # computes until data[-1]
    betas = np.ones((N, N_state))
    
    for t in range(N-2, -1, -1):
        betas[t, :] = (em[data[t+1], :].reshape(1, -1) @ (betas[t+1, :].reshape(-1, 1) * trans)).reshape(-1)
        
        # Scale to avoid underflow
        betas[t, :] /= np.max(betas[t, :])
        
    return betas

In [100]:
# 2e) EM Algo

# 1. Load data and init
chess_sequence = np.genfromtxt('chess.txt', dtype='int')

# Starting transition matrix
transMat = np.array([[0.5, 0.8], [0.5, 0.2]])

# Emission Matrix and initial hidden state (NOT updated)
initProb = np.array([1, 0])
emiProb = np.array([[0.3, 0.5], [0.6, 0], [0.1, 0.5]])

In [101]:
# 2. E-step
for it in range(50):
    # compute joint q's. Data is one long chain, 1 datapoint, of length 1000 steps. Unlike q1), now we have many hidden variables
    alphas = compute_alpha_t(transMat, emiProb, initProb, chess_sequence)
    betas = compute_sc_beta_t(transMat, emiProb, initProb, chess_sequence)
    q_joint = compute_phtt1_given_v(alphas, betas, transMat, emiProb, chess_sequence)

    # 3. M-step
    old_transMat = copy.deepcopy(transMat)

    for i in range(transMat.shape[0]):
        for j in range(transMat.shape[1]):
            transMat[i,j] = np.sum(q_joint[:, i, j])

    # Normalise
    transMat = transMat / np.sum(transMat, axis = 0)

    change = np.sum(np.abs(transMat - old_transMat))
    print("Change ", change)
    if change < 1e-8:
        print(f"Stopped at iteration {it+1}")
        break

Change  0.32691831310241476
Change  0.311196708310305
Change  0.2634981693776444
Change  0.25767264389948663
Change  0.2567090938432675
Change  0.24764146820528943
Change  0.23106177018355237
Change  0.19974883918246306
Change  0.15565703337051393
Change  0.10906970785190481
Change  0.06965619001819816
Change  0.041510176296049486
Change  0.02365113427621296
Change  0.013123002050230537
Change  0.007173724453556546
Change  0.003889619186935714
Change  0.0020996348722072677
Change  0.0011306832911071457
Change  0.0006081048342492512
Change  0.0003268247527058732
Change  0.0001755859117601316
Change  9.431428816905618e-05
Change  5.0654570586283754e-05
Change  2.7204122105509154e-05
Change  1.4609566496570636e-05
Change  7.845718343515995e-06
Change  4.213317760251578e-06
Change  2.262630461338788e-06
Change  1.2150717545456047e-06
Change  6.525136783397389e-07
Change  3.504104036100575e-07
Change  1.8817598131104507e-07
Change  1.0105348478839371e-07
Change  5.4267316848544134e-08
Chang

In [102]:
print(f"Learned transition matrix \n {transMat}")

Learned transition matrix 
 [[0.83258179 0.0723181 ]
 [0.16741821 0.9276819 ]]


In [103]:
# 2f) 
results = []; alphas = compute_alpha_t(transMat, emiProb, initProb, chess_sequence) 
for i in range(emiProb.shape[0]):
    results.append(np.sum(emiProb[i, :].reshape(-1, 1) * transMat * alphas[-1, :]))
    
results = np.array(results)
print(results)

[0.47736557 0.0679033  0.45473113]
